# LR(0) parser demo


Here we show a demo of an LR(0) parser for the language
```
S -> E$
E -> E+T
E -> T
T -> v
T -> (E)
```
If the language isn't LR(0), then sometimes using the "follow" sets will be enough
to resolve reduce-reduce conflicts, and this gives an SLR parser.

If the language isn't SLR, then we can add a lookahead character to the LR(0) items
and use the lookahead to resolve conflicts. Almost all computer languages are LR(1)
but this can generate very large parsing tables.  LALR reduces the size of these tables
by combining states which only differ in lookahead values, and these can be much smaller
tables, but not every LR(1) language is LALR, that is, some conflicts may still arise
when we combine similar states.

## Action table
Below is the action table for the LR(0) parser.
Here are the first few LR(0) states... You should draw the complete LR(0)
graph which includes finding states 3 and 4
Each of the states {0,1,2,3,4} is a set of LR(0) states.

```
0: 
S->.E$
E->.E+T
E->.T
T->.v
T->.(E)

1:
E->E.+T
S->E.$

2:
E->E+.T
T->.v
T->(E)

```
and the table is below
```
                             next symbol
state     v        +      (       )     $     F     T       E
0       r(1,T)           s(3)                     r(1,E)   s(1)
1                 s(2)                ACCEPT
2       r(1,T)           s(3)                     r(3,E)
3       r(1,T)           s(3)                     r(1,E)   s(4)
4                 s(2)          r(3,T)
```
where r(i,N) means pop the top i-1 states off of the stack to get state s
then use the table for state s and symbol N to see what action to perform
and s(k) means shift the next terminal onto the stack and go to state k

In [32]:
actions = {
    0:{'v':('reduce',1,'T'),
       '(':('shift',3),
       '+':('error',),
       ')':('error',),
       '$':('error',),
       'T':('reduce',1,'E'),
       'E':('shift',1),    
      },
    
    1:{'+':('shift',2),
       '$':('accept',),
       '(':('error',),
       ')':('error',),
       'v':('error',)
      },
    
    2:{'v':('reduce',1,'T'),
       '(':('shift',3),
       '+':('error',),
       ')':('error',),
       '$':('error',),
       'T':('reduce',3,'E'),
      },
    
    3:{'v':('reduce',1,'T'),
       '(':('shift',3), 
       '+':('error',),
       ')':('error',),
       '$':('error',),      
       'T':('reduce',1,'E'),
       'E':('shift',4)
      },
    
    4:{'+':('shift',2),
       ')':('reduce',3,'T'),
       'v':('error',),
       '(':('error',),
       '$':('error',)
      }

}
actions

{0: {'v': ('reduce', 1, 'T'),
  '(': ('shift', 3),
  '+': ('error',),
  ')': ('error',),
  '$': ('error',),
  'T': ('reduce', 1, 'E'),
  'E': ('shift', 1)},
 1: {'+': ('shift', 2),
  '$': ('accept',),
  '(': ('error',),
  ')': ('error',),
  'v': ('error',)},
 2: {'v': ('reduce', 1, 'T'),
  '(': ('shift', 3),
  '+': ('error',),
  ')': ('error',),
  '$': ('error',),
  'T': ('reduce', 3, 'E')},
 3: {'v': ('reduce', 1, 'T'),
  '(': ('shift', 3),
  '+': ('error',),
  ')': ('error',),
  '$': ('error',),
  'T': ('reduce', 1, 'E'),
  'E': ('shift', 4)},
 4: {'+': ('shift', 2),
  ')': ('reduce', 3, 'T'),
  'v': ('error',),
  '(': ('error',),
  '$': ('error',)}}

# LR(0) parser
Here is the code for an LR(0) parser which prints out a trace of its
execution. 

In [102]:
    
def parse(chars):
    stack = []
    action = ('shift',0)  # first action is to shift the 1st char and go to state 0
    states = []
    counter = 0
    
    while action[0] not in {'error','accept'} and counter<100:
        counter += 1
        stackr = stack[::-1]
        print("%3d %7s   %-20s %-20s %+20s 　%-20s"%
              (counter,action[0],str(" ".join(stackr)),"".join(chars),str(states),str(action)))
        print('-'*90)
        if action[0]=='shift':
            stack = [chars[0]]+ stack
            new_state = action[1]
            new_action = actions[new_state][chars[0]]
            action = new_action
            chars = chars[1:]
            states = [new_state]+states
        elif action[0]=='reduce':
            num_to_pop = action[1]   
            new_non_terminal = action[2]
            # instead of pushing the new non-terminal, we could push the parse tree for that nonterminal
            stack = [new_non_terminal]+stack[num_to_pop:]
            states = states[num_to_pop-1:]
            action = actions[states[0]][new_non_terminal]
    return action
                        
parse(list("v+(v+v)$"))

  1   shift                        v+(v+v)$                               [] 　('shift', 0)        
------------------------------------------------------------------------------------------
  2  reduce   v                    +(v+v)$                               [0] 　('reduce', 1, 'T')  
------------------------------------------------------------------------------------------
  3  reduce   T                    +(v+v)$                               [0] 　('reduce', 1, 'E')  
------------------------------------------------------------------------------------------
  4   shift   E                    +(v+v)$                               [0] 　('shift', 1)        
------------------------------------------------------------------------------------------
  5   shift   E +                  (v+v)$                             [1, 0] 　('shift', 2)        
------------------------------------------------------------------------------------------
  6   shift   E + (                v+v)$          

('accept',)